In [1]:
from tqdm import tqdm
import torch

from data.CRSPLoader import CRSPLoader
from loss_functions.SharpeLoss import SharpeLoss
from models.TGNNPO import TGNNPO

In [2]:
device = torch.device('cpu')
window_length = 50
step_length = 5

# load and prepare dataset
loader = CRSPLoader(load_data=True)
etf_tickers = ['SPY', 'XLF', 'XLB', 'XLK', 'XLV']
loader._update_ticker_index(ticker_list=etf_tickers)
dataset = loader.get_dataset(data=loader.select_tickers(tickers=etf_tickers), window_length=window_length, step_length=step_length)


Loading in saved CRSP data...


/Users/dylansandfelder/Documents/Work/DanielCollab/graph-portfolio-opt/src/data/CRSPLoader.py:27: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  self._load_data(self.load_path)


Generating CRSP dataset...
Generating feature matrix...


100%|██████████| 5531/5531 [00:08<00:00, 615.24it/s]


Generating target matrix...


100%|██████████| 5531/5531 [00:02<00:00, 1980.32it/s]


In [8]:
# optimization hyperparameters
learning_rate = 0.1

# how recently should we consider our model to be "trained" by?
lookback_loss_mean = 200

# training hyperparameters
epochs = 10

In [9]:
# (1) model
model = TGNNPO(node_features=loader.num_features, num_nodes=loader.num_nodes, periods=window_length).to(device)

# (2) loss function
lossfn = SharpeLoss()

# (3) training procedure
model.train()
for epoch in range(epochs + 1): 
    # optimizer resets itself between epochs
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=(learning_rate/10.0))
    loss_mean = []
    pbar = tqdm(enumerate(dataset), total=dataset.get_num_batches())
    for steps, batch in pbar:
        X_batch = batch.x
        prices_batch = batch.y
        static_edge_index = batch.edge_index
        
        optimizer.zero_grad()
        
        # predict portfolio weights
        weights_pred = model(X_batch, static_edge_index)
  
        # sharpe ratio loss
        loss = lossfn(prices_batch, weights_pred, ascent=True)
        loss_mean.append(loss.item())
        if len(loss_mean) < lookback_loss_mean:
            cur_mean = sum(loss_mean) / len(loss_mean)
        else:
            cur_mean = sum(loss_mean[(-1 * lookback_loss_mean):]) / len(loss_mean[(-1 * lookback_loss_mean):])
        pbar.set_description("Epoch: %d, sharpe (mean loss): %1.5f" % (epoch, cur_mean * -1))

        loss.backward()
        optimizer.step()  
    learning_rate = learning_rate / 2.0  

Epoch: 10, sharpe (mean loss): 1.13852: 100%|██████████| 1097/1097 [01:01<00:00, 17.73it/s]


In [ ]:
model.eval()

# Store for analysis
weights = []
prices = []

pbar = tqdm(enumerate(test_loader), total=len(test_loader))
for steps, (X_batch, prices_batch) in pbar:

    # predict portfolio weights
    weights_pred = model(X_batch, static_edge_index)

    # sharpe ratio loss
    loss = lossfn(prices_batch, weights_pred, ascent=True)

    # compute gradients and backpropagate
    loss.backward()
    optimizer.step() 
    optimizer.zero_grad()

    pbar.set_description("Test sharpe (loss): %1.5f" % (loss.item() * -1))

    # store predictions and true values
    prices.append(prices_batch)
    weights.append(weights_pred)